In [ ]:
from scipy.special import gamma, psi, polygamma
import matplotlib
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline

### Maximum likelihood оптимизация

Очень часто в машинном обучении и статистике возникает задача когда есть данные $X$, некоторое параметрическое распределение $p(x | \theta)$ и желание найти параметры $\theta$. 

Самые лучшие параметры это те которые минимизируют функцию правдоподобия: $\mathcal{L}(X, \theta)$, во многих случаях это можно сделать аналитически(почти для всех известных нам именных распределений).

В случае если нет возможности посчитать аналитически MLE-оценки приходится обращаться к итеративным методам оптимизации. Один из самых популярных способов это градиентный спуск, который, формально решает следующую задачу:

$$d \theta = \arg \max \mathcal{L}(X|\theta + d \theta), s.t. ||d\theta|| < \epsilon$$

И результатом решения этой задачи является:

$$d\theta = \frac{\nabla_\theta \mathcal{L}}{||\nabla_\theta \mathcal{L}||} \epsilon$$

Оптимально ли это? 

Мы ограничиваем наш шаг $||d\theta||$ сферой радиусом $\epsilon$ в евклидовом пространстве, но это не особо разумно. 

По этой причине мы бы хотели научиться решать следующую задачу:

$$d \theta = \arg \max \mathcal{L}(X|\theta + d \theta), s.t. KL\left( p(x| \theta) || p(x| \theta + d \theta) \right) < \epsilon$$

То есть мы хотели бы на каждом шаге оптимизации делать шаг на $\epsilon$ по метрике KL, а не по метрике $L_2$.


### KL дивергенция

KL-дивергенция выглядит следующим образом:

$$KL\left(p(x|\theta)||p(x|\theta')\right) = \int p(x|\theta) \log \frac{p(x|\theta)}{p(x|\theta')} dx $$

А теперь зададимся вопросом: как нам оптимизировать параметры так чтобы спускаться с постоянной скоростью по метрике KL?

Для этого расмотрим $\mathrm{KL}\left[ p_{\theta} || p_{\theta + d \theta} \right]$ - KL-дивергенцию от двух очень близких друг к другу распределений и разложим эту функцию по Тейлору при $d \theta \rightarrow 0$:

$$
\mathrm{KL}\left[ p_{\theta} || p_{\theta + d \theta} \right] \approx \mathrm{KL}\left[ p_{\theta} || p_{\theta} \right] + \left( \nabla_{\theta'} \mathrm{KL}\left[ p_{\theta} || p_{\theta'} \right]|_{\theta' = \theta} \right)^T d\theta + \frac{1}{2} d \theta^T \left( \nabla_{\theta'}^2 \mathrm{KL}\left[ p_{\theta} || p_{\theta'} \right] |_{\theta' = \theta} \right) d \theta
$$

Посчитаем второй член:

$$
\nabla_{\theta'} \mathrm{KL}\left(p(x|\theta)||p(x|\theta')\right) |_{\theta' = \theta}  = - \nabla_{\theta'} \mathrm{E}_{ p(x|\theta)} \log p(x|\theta') |_{\theta' = \theta} = \\
= \int \nabla \log p(x|\theta) p(x|\theta) dx = 0
$$


Посчитаем второй член:

А теперь возьмём вторую производную.

$$
\nabla_{\theta'}^2 \mathrm{KL}\left(p(x|\theta)||p(x|\theta')\right)  = - \int p(x|\theta) \nabla_{\theta'}^2 \log p(x | \theta') dx
$$

Вторая производная по вектору параметров от $\log p(x|\theta')$ это матрица Гессе. 

Если расмотреть её в точке $\theta=\theta'$, то результат будет следующий(без вывода):

$$
\nabla_{\theta'}^2 \mathrm{KL}\left(p(x|\theta)||p(x|\theta')\right) = - E \left[ \mathrm{H}_{\log p(x | \theta)} \right]
$$


###### Теперь мы готовы вернуться к исходному разложению:

$$
\mathrm{KL}\left[ p_{\theta} || p_{\theta + d \theta} \right] \approx - \frac{1}{2} d \theta^T E \left[ \mathrm{H}_{\log p(x | \theta)} \right] d \theta 
$$


##### Ещё одно усилие :)

Теперь мы готовы решить изначальную задачу:

$$d \theta = \arg \max \mathcal{L}(X|\theta + d \theta), s.t. KL\left( p(x| \theta) || p(x| \theta + d \theta) \right) < \epsilon$$

Перепишем её через теорему ККТ:

$$d \theta = \arg \max \mathcal{L}(X|\theta + d \theta) + \lambda \left( \mathrm{KL}\left[ p_{\theta} || p_{\theta + d \theta} \right]  - \epsilon \right)$$

Разложим log-likelihood до линейного члена, а KL до квадратичного:

$$
d \theta \approx \arg \max  \mathcal{L}(\theta) + \nabla_\theta^T d \theta -  \frac{1}{2} \lambda d \theta^T E \left[ \mathrm{H}_{\log p(x | \theta)} \right] d \theta  - \lambda \epsilon
$$

Возьмём производную по $d \theta$ от правой части и приравняем нулю:

$$
0 = \nabla_\theta \mathcal{L} - \lambda E \left[ \mathrm{H}_{\log p(x | \theta)} \right] d \theta
$$

$$
d \theta = \frac{1}{\lambda} E \left[ \mathrm{H}_{\log p(x | \theta)} \right]^{-1} \nabla_\theta \mathcal{L}
$$


### Связь KL-дивергенции и матрицы Фишера

Гессиан log-likelihood функции это одна из форм записи матрицы Фишера:

$$I_n = - E \left[ \mathrm{H}_{\log p(x | \theta)} \right]$$

Таким образом, матрица Фишера, гессиан log-likelihood функции и гессиан KL-дивергенции это одно и тоже.

### Расчёт матрицы Фишера

Более удобный способ расчёта матрицы Фишера:

$$I_n = E\left[ \left( \frac{\partial \log p(\vec{x}|\vec{\theta})}{\partial \vec{\theta}} \right) 
\left( \frac{\partial \log p(\vec{x}|\vec{\theta})}{\partial \vec{\theta}}\right)^T  \right]$$


Производную мы посчитали выше. А как нам посчитать матожидание?

Усреднение по элементам выборки есть приближение среднего,поэтому самый простой способ это посчитать матрицу Фишера для каждого элемента нашей выборки и усреднить.

# Задача


### 0. Вступление

Давайте вспомним задачу про генотипы и относительную приспособленность.

"Относительную приспособленность" определяют среднее количество выживших представителей одного генотипа к среднему количеству выживших представителей другого генотипа. 

Eyre-Walker (2006) ( https://www.ncbi.nlm.nih.gov/pubmed/16547091 ) предположили, что функция преспособленности принадлежит семейству гамма-распределений случайных величин. 

Они проверили своё предположение для популяций людей с мутацией `deleterious amino acid`. 

Они получили следующие оценки на параметры гамма-распределения:
 
$$\hat{\alpha} = 0.23,~~~ \hat{\beta} = 5.35$$

$$\Gamma(x | \alpha, \beta) = \frac{\beta^\alpha}{\Gamma(\alpha)} x^{\alpha - 1} \exp(- \beta x)$$

### 1. Завязка сюжета и actual task

На семинаре мы забили на непосредственную оптимизацию нашей функции и использовали `.fit`-метод, который предоставлял интерфейс в `scipy`-модуле.

Теперь мы попробуем сами прооптимизировать вот это вот всё с помощью градиентного спуска и натурального градиентного спуска.

А так же исследуем как введение априорных распределений влияет на результат работы алгоритма.

### 2. Подробное описание задания

1. Выпишите производные по $\alpha, \beta$ для логарифма правдоподобия гамма-распределения($\Gamma(x | \alpha, \beta)$);
2. Выпишите матрицу Фишера для гамма-распределения;
3. (1 балл) Реализуйте градиентный спуск и натуральный градиентный спуск и сравните финальное качество и скорость сходимости(i.e. постройте кривые зависимости логарифма правдоподобия от номера итерации).
    1. При решении п. 2 и п. 3 полезно почитать главу 9 отсюда: http://www.sherrytowers.com/mle_introduction.pdf
4. Придумайте приорное распределение на $\alpha$, $\beta$: $p(\alpha, \beta)$. Во-первых, подумайте(и напишите!) какой биологический смысл эти параметры несут. Используя своё понимание, придумайте подходящее априорное распределения.
5. Выведите самую неинформативную априорную вероятность Джеффриса для гамма-распределения($p(\alpha, \beta) = \sqrt{|I(\alpha, \beta)|}$): https://ru.wikipedia.org/wiki/Априорная_вероятность_Джеффриса
    
    Из названия следует, что эта априорная вероятность является самой неинформативной(спс кэп), например, для $\mu$ в $\mathcal{N}(x | \mu, \sigma^2)$ априорной вероятностью Джеффриса будет равномерное распределение на всей числовой прямой.
    1. Здесь вам пригодится матрица Фишера, которую вы посчитали в п. 2
6. (1 балл) Выпишите производные по $\alpha, \beta$ и матрицу Фишера для распределений: $p(\alpha, \beta | x) = 
\frac{\Gamma(x | \alpha, \beta) p(\alpha, \beta)}{p(x)}$ с априорным распределением выбранным вами и с априорным распределением Джеффриса.
7. Примените алгоритмы градиентного спуска и натурального градиентного спуска из п. 3 для оценки параметров $\alpha, \beta$ с вашей приорой и с приорой Джеффриса. 
8. (1 балл) В итоге у вас будет 4 графика сходимости log-likelihood от номера итераций. 

    К каким $\alpha$ и $\beta$ сошёлся алгоритм? Какие выводы вы можете сделать?
    
    Чем больше выводов и наблюдений вы сделаете -- тем лучше :)
    
    При экспериментах попробуйте разные инициализации $\alpha$ и $\beta$ и разные learning rates. Посмотрите к чему приводит их вариация.
    

    
### 3. Заготовки кода

(Вы можете их как использовать, так и не использовать)

In [ ]:
class GammaDistributionOracle:
    def __init__(self, a, b):
        self._a = a
        self._b = b
        
    def update_parameters(self, a, b):
        self._a = a
        self._b = b
    
    def p(self, x):
        """
        Probability to observe sample x
        """
        return (np.power(self._b, self._a) / gamma(alpha)) * x**(self._a - 1) * np.exp(-self._b * x)
    
    def __call__(self, x):
        return self.p(x=x)
    
    def log_p(self, x):
        """
        Log-probability to observe sample x
        """
        NotImplementedError('log_p is not implemented')
        
    def grad(self, x):
        """
        Derivative of log-probability [\alpha, \beta] at x
        """
        NotImplementedError('grad is not implemented')
    
    def fisher_matrix(self):
        """
        Fisher matrix for Gamma distribution
        """
        NotImplementedError('fisher_matrix is not implemented')

In [ ]:
class PriorOracle:
    def __init__(self, a, b):
        self._a = a
        self._b = b
        
    def update_parameters(self, a, b):
        self._a = a
        self._b = b
    
    def p(self):
        """
        Probability
        """
        raise NotImplementedError('p is not implemented')
    
    def __call__(self):
        return self.p()
    
    def log_p(self):
        """
        Log-probability to observe sample x
        """
        raise NotImplementedError('log_p is not implemented')
        
    def grad(self):
        """
        Derivative of log-probability [\alpha, \beta] at x
        """
        raise NotImplementedError('grad is not implemented')
    
    def fisher_matrix(self):
        """
        Fisher matrix for Gamma distribution
        """
        raise NotImplementedError('fisher_matrix is not implemented')

In [ ]:
from collections import defaultdict
def gradient_descent(X, oracle, prior=None, epochs=100, lr=1e-3):
    """
    Perfoms simple gradient descent with known oracle and prior
    """
    history = defaultdict(list)
    for epoch in range(epochs):
        raise NotImplementedError('Gradient descent is not implemented')
        gradient = ...
        if prior: 
            gradient += ...
        a = ...
        b = ...
        
        # update parameters of oracle
        oracle.update_parameters(a=a, b=b)
        # update parameters of prior
        if prior: prior.update_parameters(a=a, b=b)
        history['log-likelihood'].append(oracle.log_p(X).mean())
        if prior: history['log-prior'].append(prior.log_p())
        history['a'].append(a)
        history['b'].append(b)
        
    return history

def natural_gradient_descent(X, oracle, prior=None, epochs=100, lr=1e-3):
    """
    Perfoms simple gradient descent with known oracle and prior
    """
    history = defaultdict(list)
    for epoch in range(epochs):
        NotImplementedError('Natural gradient descent is not implemented')
        gradient = ...
        fisher_matrix = ...
        if prior: 
            gradient = ...
            fisher_matrix = ...
        gradient = ...
        a = ...
        b = ...
        
        # update parameters of oracle
        oracle.update_parameters(a=a, b=b)
        # update parameters of prior
        if prior: prior.update_parameters(a=a, b=b)
        history['log-likelihood'].append(oracle.log_p(X).mean())
        if prior: history['log-prior'].append(prior.log_p())
        history['a'].append(a)
        history['b'].append(b)
        
    history['a'] = np.array(history['a'])
    history['b'] = np.array(history['b'])
    history['log-likelihood'] = np.array(history['log-likelihood'])
    if "log-prior" in history:
        history['log-prior'] = np.array(history['log-prior'])
    return history

In [ ]:
def plotter(history):
    f, ax = plt.subplots(1, 3, figsize=(16, 6))
    ax[0].plot(history['log-likelihood'], label='Log-likelihood')
    if "log-prior" in history:
        ax[0].plot(history['log-prior'], label='Log-prior')
        ax[0].plot(history['log-likelihood'] + history['log-prior'], label='Log-likelihood')
    ax[1].plot(history['a'], label=r'$\alpha$')
    ax[2].plot(history['b'], label=r'$\beta$')
    plt.legend()
    plt.plot()

### 4. Пайплайн запуска

In [ ]:
from scipy.stats import gamma as gamma_dist
np.random.seed(1337)
alpha = 0.23
beta = 5.35
# генерируем выборку
X = gamma_dist.rvs(scale=1 / beta, a=alpha, size=100)

In [ ]:
alpha_init = 1
beta_init = 0.1

In [ ]:
oracle = GammaDistributionOracle(a=alpha_init, b=beta_init)
prior = PriorOracle(a=alpha_init, b=beta_init) # prior = None, если не используется приора
history = gradient_descent(oracle=oracle, X=X, prior=prior, epochs=10000)
plotter(history=history)
print(history['log-likelihood'][-1], history['a'][-1], history['b'][-1])

In [ ]:
oracle = GammaDistributionOracle(a=alpha_init, b=beta_init)
prior = PriorOracle(a=alpha_init, b=beta_init) # prior = None, если не используется приора
history = natural_gradient_descent(oracle=oracle, prior=prior, X=X, epochs=10000) 
plotter(history=history)
print(history['log-likelihood'][-1], history['a'][-1], history['b'][-1])